In [29]:
!pip install -q -U  accelerate
# git+https://github.com/huggingface/accelerate

In [3]:
pip install -q -U transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -q -U bitsandbytes einops sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [5]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, BitsAndBytesConfig


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [7]:
model_id = "decapoda-research/llama-7b-hf"

In [8]:
import torch

In [9]:
bnbConfig = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [10]:
model = LlamaForCausalLM.from_pretrained(model_id,
                                         load_in_4bit=True,
                                         torch_dtype=torch.bfloat16,
                                         quantization_config=bnbConfig,
                                         device_map="auto")

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [11]:
generation_config = GenerationConfig(
    temperature=0.01,
    top_p=0.9,
    typical_p=0.9,
    repetition_penalty=2.0,
    encoder_repetition_penalty=2.0,
    top_k=40,
    renormalize_logits=True,
    do_sample=True,
    num_beams=2,
    num_return_sequences=1,
    remove_invalid_values=True
)

In [12]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
    

In [14]:
pip install -q -U git+https://github.com/huggingface/peft.git

Note: you may need to restart the kernel to use updated packages.


In [15]:
from peft import PeftModel

In [16]:
modelPeft = PeftModel.from_pretrained(model, "0sunfire0/Llama_7B_Test08")

In [17]:
modelPeft

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=Fa

In [18]:
tokenizer = LlamaTokenizer.from_pretrained(model_id)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [19]:
custom_prompt = "I want to invest in US Treasury Securities. I am looking for long term investments but I want minimal risk. I prefer stable income because I am already 42 years old."

PROMPT =f'''Below is an instruction that describes a task. Write a response in detail that appropriately completes the request.


### Instruction:
{custom_prompt}
### Response:
'''

In [20]:
PROMPT

'Below is an instruction that describes a task. Write a response in detail that appropriately completes the request.\n\n\n### Instruction:\nI want to invest in US Treasury Securities. I am looking for long term investments but I want minimal risk. I prefer stable income because I am already 42 years old.\n### Response:\n'

In [21]:
inputs = tokenizer(
    PROMPT,
    return_tensors="pt",
)
input_ids = inputs["input_ids"].cuda()

In [22]:
generation_output = modelPeft.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=False,
    max_new_tokens=700,
)

In [23]:
modelPeft

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=Fa

In [25]:
PROMPT

'Below is an instruction that describes a task. Write a response in detail that appropriately completes the request.\n\n\n### Instruction:\nI want to invest in US Treasury Securities. I am looking for long term investments but I want minimal risk. I prefer stable income because I am already 42 years old.\n### Response:\n'

In [24]:
for s in generation_output.sequences:
    print(tokenizer.decode(s))    

 ⁇  Below is an instruction that describes a task. Write a response in detail that appropriately completes the request.


### Instruction:
I want to invest in US Treasury Securities. I am looking for long term investments but I want minimal risk. I prefer stable income because I am already 42 years old.
### Response:
U.S. Treasury securities are generally considered to be among the safest investments available. They are backed by the U.S. government, which has a strong credit rating and a history of meeting its debt obligations. That being said, long-term investments always carry some level of risk, regardless of the nature of the investment.

If safety and minimal risk are your primary concerns, you may consider the following options:

1. U.S. Government Securities: U.S. Treasury securities are the go-to choice for safe long-term investments. They include Treasury bonds, Treasury notes, Treasury bills, and Treasury Inflation-Protected Securities (TIPS). These securities are issued wit